In [1]:
import pandas as pd
from pathlib import Path
import util

pd.set_option('display.float_format', '{:,.1f}'.format)


In [2]:
# trip data
trip = pd.read_csv(util.output_path / 'agg/dash/person_trips.csv')
# person data
person = pd.read_csv(util.output_path / 'agg/dash/person_geog.csv')
# vmt data
vmt = pd.read_csv(util.output_path / 'agg/dash/person_vmt.csv')

# list of equity geographies
equity_geogs = util.summary_config['hh_equity_geogs']
# not_equity_geogs = ["NOT in " + item for item in equity_geogs]

In [3]:
# TRIPS
df_trip = trip.copy()
# add home RGC
df_trip['is_rgc'] = 'Not in RGC'
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
# add trip type
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'

# PERSONS
df_person = person.copy()
# add home RGC
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# VMT
df_vmt = vmt.copy()
# add home RGC
df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

# Select only walk and bike trips
df_vmt_bp = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])].copy()
# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)].copy()

# # not in equity geography
# df_person[not_equity_geogs] = 1 - df_person[equity_geogs]
# df_trip[not_equity_geogs] = 1 - df_trip[equity_geogs]
# df_vmt[not_equity_geogs] = 1 - df_vmt[equity_geogs]
# df_vmt_bp[not_equity_geogs] = 1 - df_vmt_bp[equity_geogs]

# total population by equity geography
equity_geogs_population = df_person[equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
equity_geogs_population.columns = ['Equity Group', 'psexpfac']

# total population by "NOT in" equity geography
# not_equity_geogs_population = df_person[not_equity_geogs].apply(lambda x: x * df_person['psexpfac']).sum().reset_index()
# not_equity_geogs_population.columns = ['Equity Group', 'psexpfac']

## Trips per Day by Resident

In [4]:
def trips_per_day(geog, map=False): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total trips by geography
    df2 = df_trip.groupby([geog], as_index=False)['trexpfac'].sum().set_index(geog)

    # total trips by trip type and geography
    df3 = df_trip.groupby([geog, 'trip_type'], as_index=False)['trexpfac'].sum().set_index(geog)
    df3 = df3.pivot(columns='trip_type', values='trexpfac')

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)
    df = df.merge(df3, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # regional totals
    df.loc['Region', ['Work','Non-Work','psexpfac','trexpfac']] = df[['Work','Non-Work','psexpfac','trexpfac']].sum()

    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']
    df['Work Trips per Day'] = df['Work']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work']/df['psexpfac']


    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [5]:
df = trips_per_day('hh_county')
df = df[df.index != 'Outside Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.6,3.4,3.9
Kitsap,0.5,3.2,3.7
Pierce,0.4,3.2,3.6
Snohomish,0.5,3.3,3.8
Region,0.5,3.3,3.8


In [6]:
trips_per_day('is_rgc')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
is_rgc,,,
In RGC,0.6,3.4,4.0
Not in RGC,0.5,3.3,3.8
Region,0.5,3.3,3.8


In [7]:
trips_per_day('hh_rgc')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.5,3.4,3.9
Bellevue,0.7,3.4,4.1
Bothell Canyon Park,0.5,3.4,3.9
Bremerton,0.5,3.4,3.9
Burien,0.5,3.4,3.9
Everett,0.5,3.4,3.9
Federal Way,0.5,3.4,3.8
Greater Downtown Kirkland,0.6,3.4,3.9
Issaquah,0.6,3.3,3.9


In [8]:
trips_per_day('hh_rg_proposed')

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
Cities and Towns,0.5,3.2,3.7
Core Cities,0.5,3.3,3.8
High Capacity Transit Communities,0.5,3.3,3.8
Metropolitan Cities,0.6,3.4,4.0
Rural Areas,0.4,3.1,3.5
Urban Unincorporated Areas,0.4,3.3,3.6
Region,0.5,3.3,3.8


### Equity Focus Areas

In [9]:
trips_per_day('hh_efa_poc', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_poc,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.3,3.9
Higher Share of Equity Population,0.5,3.4,3.9
Region,0.5,3.3,3.8


In [10]:
trips_per_day('hh_efa_pov200', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_pov200,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.3,3.8
Region,0.5,3.3,3.8


In [11]:
trips_per_day('hh_efa_lep', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_lep,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.3,3.9
Higher Share of Equity Population,0.5,3.4,3.9
Region,0.5,3.3,3.8


In [12]:
trips_per_day('hh_efa_dis', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_dis,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.3,3.8
Region,0.5,3.3,3.8


In [13]:
trips_per_day('hh_efa_older', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_older,,,
Below Regional Average,0.5,3.3,3.8
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.3,3.7
Region,0.5,3.3,3.8


In [14]:
trips_per_day('hh_efa_youth', map=True)

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_efa_youth,,,
Below Regional Average,0.5,3.3,3.9
Above Regional Average,0.5,3.3,3.8
Higher Share of Equity Population,0.5,3.2,3.7
Region,0.5,3.3,3.8


## Miles Driven per Day by Resident

In [15]:
# pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(util.output_path / 'agg/dash/person_vmt.csv')
df_person = person.copy()

df_vmt['is_rgc'] = 'Not in RGC'
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
df_person['is_rgc'] = 'Not in RGC'
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'


# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [16]:
def miles_per_day(geog, map=False): 
    """
    Calculate trips per day by geography
    """

    # total population by geography
    df1 = df_person.groupby([geog], as_index=False)['psexpfac'].sum().set_index(geog)

    # total miles by geography
    df2 = df_vmt.groupby([geog], as_index=False)['travdist_wt'].sum().set_index(geog)

    # Merge the dataframes
    df = df1.merge(df2, left_index=True, right_index=True)

    if map:
        df.index = df.index.astype('int').map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

    # regional totals
    df.loc['Region', ['psexpfac','travdist_wt']] = df[['psexpfac','travdist_wt']].sum()

    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

In [17]:
df = miles_per_day('hh_county')
df = df[df.index != 'Outside Region']
df

,Average Miles per Person
hh_county,
King,11.5
Kitsap,13.1
Pierce,13.9
Snohomish,14.4
Region,12.7


In [18]:
miles_per_day('is_rgc')

,Average Miles per Person
is_rgc,
In RGC,4.5
Not in RGC,14.2
Region,12.7


In [19]:
miles_per_day('hh_rgc')

,Average Miles per Person
hh_rgc,
Auburn,7.3
Bellevue,2.3
Bothell Canyon Park,13.1
Bremerton,3.7
Burien,8.7
Everett,3.6
Federal Way,7.7
Greater Downtown Kirkland,9.8
Issaquah,11.1


In [20]:
miles_per_day('hh_rg_proposed')

,Average Miles per Person
hh_rg_proposed,
Cities and Towns,17.6
Core Cities,12.2
High Capacity Transit Communities,13.6
Metropolitan Cities,7.5
Rural Areas,22.9
Urban Unincorporated Areas,15.9
Region,12.7


In [21]:
df = pd.DataFrame()
for name, col in {
    "People of Color": "hh_efa_poc",
    "Income": "hh_efa_pov200",
    "LEP": "hh_efa_lep",
    "Disability": "hh_efa_dis",
    "Older Adults": "hh_efa_older",
    "Youth": "hh_efa_youth"
}.items():
    df[name] = miles_per_day(col, map=True)
df

,People of Color,Income,LEP,Disability,Older Adults,Youth
hh_efa_poc,,,,,,
Below Regional Average,14.5,14.0,13.4,13.2,11.8,10.6
Above Regional Average,10.9,12.0,12.0,13.1,13.8,14.9
Higher Share of Equity Population,10.8,9.4,11.1,10.5,13.7,16.2
Region,12.7,12.7,12.7,12.7,12.7,12.7


## Delay

In [22]:
pd.options.display.float_format = '{:0,.1f}'.format

def delay_per_person(geog):

    df_person = pd.read_csv(util.output_path / 'agg/dash/person_geog.csv')
    df_person['is_rgc'] = 'Not in RGC'
    df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'

    df = pd.read_csv(util.output_path / 'agg/dash/trip_time_total.csv')
    df['is_rgc'] = 'Not in RGC'
    df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df = df[(df['mode'].isin(['SOV','HOV2','HOV3+'])&(df['dorp']==1))]
    df = df.groupby(geog).sum()[['travtime_wt']]

    df2 = pd.read_csv(util.output_path / 'agg/dash/trip_sov_ff_time.csv')
    df2['is_rgc'] = 'Not in RGC'
    df2.loc[df2['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 'In RGC'
    df2 = df2[(df2['mode'].isin(['SOV','HOV2','HOV3+'])&(df2['dorp']==1))]
    df2 = df2.groupby(geog).sum()[['sov_ff_time_wt']]
    df = df2.merge(df, on=geog)

    # Hours of delay from travel time (in min)
    df['Total Delay Hours'] = (df['travtime_wt'] - df['sov_ff_time_wt'])/60
    # Set any negative delay to 0
    df.loc[df['Total Delay Hours'] < 0, 'Total Delay Hours'] = 0

    df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = df.merge(df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Minutes of Delay per Person'] = df['Total Delay Hours']/df['psexpfac']*60

    df['Annual Hours of Delay per Person'] = df['Average Minutes of Delay per Person']*util.summary_config['weekday_to_annual']/60

    df[['Total Delay Hours',
        'Annual Hours of Delay per Person']] = df[['Total Delay Hours', 'Annual Hours of Delay per Person']].astype(int).map('{:,}'.format)


    return df[['Total Delay Hours','Average Minutes of Delay per Person','Annual Hours of Delay per Person']]

df = delay_per_person('hh_county')
df = df[df.index != 'Outside Region']
df

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_county,,,
King,"123,825",2.5,13
Kitsap,"8,025",1.4,7
Pierce,"62,382",3.0,16
Snohomish,"69,039",3.4,18
Region,"263,272",2.7,14


In [23]:
delay_per_person('is_rgc')

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
is_rgc,,,
In RGC,"13,838",0.9,4
Not in RGC,"249,434",3.1,16
Region,"263,272",2.7,14


In [24]:
df = delay_per_person('hh_rgc')

df

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_rgc,,,
Auburn,285,1.4,7
Bellevue,607,0.5,2
Bothell Canyon Park,894,4.4,23
Bremerton,63,0.3,1
Burien,655,2.0,10
Everett,469,0.6,3
Federal Way,382,1.5,7
Greater Downtown Kirkland,607,3.0,16
Issaquah,178,2.3,12


In [25]:
delay_per_person('hh_rg_proposed')

,Total Delay Hours,Average Minutes of Delay per Person,Annual Hours of Delay per Person
hh_rg_proposed,,,
Cities and Towns,"28,364",3.6,19
Core Cities,"68,309",3.0,15
High Capacity Transit Communities,"73,224",3.4,18
Metropolitan Cities,"43,865",1.4,7
Rural Areas,"33,965",3.5,18
Urban Unincorporated Areas,"15,543",4.0,21
Region,"263,272",2.7,14
